## Cluster Analysis 

### Create new dataset with relevant features: 
- Average time taken for each problem (d) 
- average accuracy (d)
- Number of times a user upgrades (d)
- Number of time a user downgrades (d)
- Average time between each exercise 
- points (d)
- user grade (d) 
- teacher count (d)
- student count (d)
- class count (d)
- belongs to count (d) 
- badges count (d)

In [1]:
# Import libraries 
import pandas as pd 
import numpy as np 
import re  
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
info_content = pd.read_csv('C:/Users/AdminCOOP/desktop/BT4103/dataset/info_content.csv')
info_userdata = pd.read_csv('C:/Users/AdminCOOP/desktop/BT4103/dataset/info_UserData.csv')
log_problem = pd.read_csv('C:/Users/AdminCOOP/desktop/BT4103/dataset/Log_Problem.csv')

In [3]:
info_userdata.head()

,uuid,gender,points,badges_cnt,first_login_date_TW,user_grade,user_city,has_teacher_cnt,is_self_coach,has_student_cnt,belongs_to_class_cnt,has_class_cnt
0,Y2RcCdmUJAYPUAIDElo4nE9KrkLLFzUIRdexG+ipaZQ=,NaN,18300,1,2019-01-24,1,kh,0,False,0,0,0
1,lw/Rchfvl9q1UDaQRmeE6QJDJeXAK7nt56RvUvqxD/8=,NaN,6468,0,2019-01-24,1,ntpc,1,False,0,1,0
2,ncVYyCw3osV77X9M+4NbI7LvBR5UiB4ix6Ca+baQArA=,male,4703,0,2019-01-24,1,tp,0,False,0,0,0
3,J7Tbo1x2WtRpPuXeX7lWT9tkzWlSJeubl8UWjNmHh+4=,NaN,15525,1,2019-01-24,2,ntpc,0,False,0,0,0
4,qijKzROzz1LmCaCxHJ3mOBOtjW/q4kW80tnpPmXHVYQ=,NaN,7945,0,2019-01-24,2,km,1,False,0,1,0


In [4]:
log_problem.head()

,timestamp_TW,uuid,ucid,upid,problem_number,exercise_problem_repeat_session,is_correct,total_sec_taken,total_attempt_cnt,used_hint_cnt,is_hint_used,is_downgrade,is_upgrade,level
0,2019-05-26 21:00:00 UTC,FLy+lviglNR5Y1l0Xiijnl6QHySBcpKHJLCtQ6ogm2Q=,KDOmuTrY/IJzDP4kIgIYCBiGyTymsJ8Iy4cDB35WGYg=,Vbs92l4JmdiWkUEm/iahxnUTaac2oN1IlUtXB7JcfoE=,18,2,True,33,1,0,False,False,True,3
1,2019-05-17 16:30:00 UTC,+Gqj2nalc6M9fusyVECTC0AN7UQdDQTXESIuElkDltU=,COZ39Wo+uIUO2s7c2VGEHjJf6Vx0xifxVAiaeHtaTdk=,Ek+pIeHNNoEo0tGEq91eBcBmGgy3+A5RWhpj95zTyHM=,4,1,True,8,1,0,False,NaN,NaN,0
2,2019-05-15 19:15:00 UTC,6D5QN8j8ng/VR74ES3A0zqAj0bIFFyaKjKEj8ZyXjQ8=,TwyqyV1uJYlDAX8wX/PtTCVZEBo/APIVfTzzleGkNCQ=,1MBa2f5Qog4JBoAuUfJf0fxeJctdEirAqKgfsg246eI=,9,1,True,17,1,0,False,NaN,NaN,0
3,2019-05-05 14:45:00 UTC,GgTZuCqZXObthtK6GAwqvlHrTMm5pKHWeezQxL/pcKc=,tBo6ECyT8IlKAM8UhQHWkqv92PRLcSiwuerfC7vNX+w=,kdMy2nG+QVMjPkuaMEWs0yV/sYZVoG1vm7zM0fCy+qk=,2,1,True,10,1,0,False,NaN,NaN,0
4,2019-05-14 16:45:00 UTC,JMNKWoU0CkMSzgQ8bCnmCYlD8jEzAVge3lHMYLXKM2g=,vVpSKAMQbTMvtdERR0ksOeRmmaFt0R210t4Z//0RpPA=,jjPR8fmkLSFoCQQYB4g6kI8mgdcK3sKtMirKUvfmZIk=,6,1,True,98,1,0,False,NaN,NaN,0


In [5]:
new_df = info_userdata.drop(['gender', 'first_login_date_TW','user_city','is_self_coach'], axis=1)

In [6]:
new_df.head()

,uuid,points,badges_cnt,user_grade,has_teacher_cnt,has_student_cnt,belongs_to_class_cnt,has_class_cnt
0,Y2RcCdmUJAYPUAIDElo4nE9KrkLLFzUIRdexG+ipaZQ=,18300,1,1,0,0,0,0
1,lw/Rchfvl9q1UDaQRmeE6QJDJeXAK7nt56RvUvqxD/8=,6468,0,1,1,0,1,0
2,ncVYyCw3osV77X9M+4NbI7LvBR5UiB4ix6Ca+baQArA=,4703,0,1,0,0,0,0
3,J7Tbo1x2WtRpPuXeX7lWT9tkzWlSJeubl8UWjNmHh+4=,15525,1,2,0,0,0,0
4,qijKzROzz1LmCaCxHJ3mOBOtjW/q4kW80tnpPmXHVYQ=,7945,0,2,1,0,1,0


In [7]:
pivot_upgrades = log_problem.pivot_table(index='uuid', values='is_upgrade', aggfunc='sum')
pivot_downgrades = log_problem.pivot_table(index='uuid', values='is_downgrade', aggfunc='sum')

In [8]:
pivot_correct = log_problem.pivot_table(values='is_correct', index='uuid', aggfunc='sum')
pivot_attempted = log_problem.pivot_table(values='is_correct', index='uuid', aggfunc='count')

student_accuracies = pivot_correct / pivot_attempted
student_accuracies

,is_correct
uuid,
++5bdNp/LZvGenJ8Brp4n2SfS9d4pu4qA7cF7FQW7hk=,1.000000
++9EkR6syMGk44XpyhOj40cg9xiXwCLS/TwEy+ujrL0=,0.103448
++E4TrlDYvGtPBg1edhkLXLEEbnfiAgAamPQ33vpW8M=,0.700000
++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,0.560976
++GobOSWqrsaxoRg1bMN+T6biIJcgBXwuOH/ddq3DiU=,0.883871
...,...
zztwLsjf44Uzumou84qh04wObpugkIllLCIp/mgoMu8=,0.884615
zzueVRV14tXIEgW4ixiqmtPNaFYz4MIifbMM/H2tmqA=,0.666667
zzujjvdKzb4wL8cFKdF3Of5gg2XHzIDQCZ2TfaYpSNs=,0.552743


In [14]:
pivot_time = log_problem.pivot_table(values='total_sec_taken',index='uuid', aggfunc='sum')
pivot_time['average_time_taken'] = pivot_time['total_sec_taken'] / pivot_attempted['is_correct']

In [15]:
pivot_time

,total_sec_taken,average_time_taken
uuid,,
++5bdNp/LZvGenJ8Brp4n2SfS9d4pu4qA7cF7FQW7hk=,302,60.400000
++9EkR6syMGk44XpyhOj40cg9xiXwCLS/TwEy+ujrL0=,428,14.758621
++E4TrlDYvGtPBg1edhkLXLEEbnfiAgAamPQ33vpW8M=,898,89.800000
++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,1550,37.804878
++GobOSWqrsaxoRg1bMN+T6biIJcgBXwuOH/ddq3DiU=,4943,31.890323
...,...,...
zztwLsjf44Uzumou84qh04wObpugkIllLCIp/mgoMu8=,3388,130.307692
zzueVRV14tXIEgW4ixiqmtPNaFYz4MIifbMM/H2tmqA=,2358,87.333333
zzujjvdKzb4wL8cFKdF3Of5gg2XHzIDQCZ2TfaYpSNs=,11117,46.907173


In [13]:
pivot_attempted

,is_correct
uuid,
++5bdNp/LZvGenJ8Brp4n2SfS9d4pu4qA7cF7FQW7hk=,5
++9EkR6syMGk44XpyhOj40cg9xiXwCLS/TwEy+ujrL0=,29
++E4TrlDYvGtPBg1edhkLXLEEbnfiAgAamPQ33vpW8M=,10
++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,41
++GobOSWqrsaxoRg1bMN+T6biIJcgBXwuOH/ddq3DiU=,155
...,...
zztwLsjf44Uzumou84qh04wObpugkIllLCIp/mgoMu8=,26
zzueVRV14tXIEgW4ixiqmtPNaFYz4MIifbMM/H2tmqA=,27
zzujjvdKzb4wL8cFKdF3Of5gg2XHzIDQCZ2TfaYpSNs=,237


In [36]:
#Find the number of unique exercises that the user take 
pivot_exercises = log_problem.pivot_table(index='uuid', values='ucid', aggfunc=pd.Series.nunique)
pivot_exercises

,ucid
uuid,
++5bdNp/LZvGenJ8Brp4n2SfS9d4pu4qA7cF7FQW7hk=,1
++9EkR6syMGk44XpyhOj40cg9xiXwCLS/TwEy+ujrL0=,1
++E4TrlDYvGtPBg1edhkLXLEEbnfiAgAamPQ33vpW8M=,1
++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,6
++GobOSWqrsaxoRg1bMN+T6biIJcgBXwuOH/ddq3DiU=,25
...,...
zztwLsjf44Uzumou84qh04wObpugkIllLCIp/mgoMu8=,5
zzueVRV14tXIEgW4ixiqmtPNaFYz4MIifbMM/H2tmqA=,4
zzujjvdKzb4wL8cFKdF3Of5gg2XHzIDQCZ2TfaYpSNs=,28


In [47]:
pivot_unique_time = log_problem.pivot_table(index='uuid',values='timestamp_TW', aggfunc = pd.Series.nunique)
pivot_unique_time

,timestamp_TW
uuid,
++5bdNp/LZvGenJ8Brp4n2SfS9d4pu4qA7cF7FQW7hk=,1
++9EkR6syMGk44XpyhOj40cg9xiXwCLS/TwEy+ujrL0=,2
++E4TrlDYvGtPBg1edhkLXLEEbnfiAgAamPQ33vpW8M=,2
++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,5
++GobOSWqrsaxoRg1bMN+T6biIJcgBXwuOH/ddq3DiU=,13
...,...
zztwLsjf44Uzumou84qh04wObpugkIllLCIp/mgoMu8=,6
zzueVRV14tXIEgW4ixiqmtPNaFYz4MIifbMM/H2tmqA=,4
zzujjvdKzb4wL8cFKdF3Of5gg2XHzIDQCZ2TfaYpSNs=,28


In [49]:
pivot_exercises = pivot_exercises.merge(pivot_unique_time,on='uuid',how='inner')

In [39]:
def get_average_time(row):
    '''
    helper function to populate the time taken between each exercise
    '''
    if row['ucid'] == 1 | row['timestamp_TW'] == 1: 
        return 0 
    else: 
        
        time_spent = 0  
        #get the list of exercises from the user & sort by exercise then time  
        temp_df = log_problem.loc[log_problem['uuid'] == row['uuid']].sort_values(by=['ucid','timestamp_TW'])
        
        #filter out the first and last occurence at that time stamp  
        temp_df_2 = temp_df.groupby('timestamp_TW').agg(['first', 'last']).stack().reset_index('timestamp_TW')
        
        #drop first and last row 
        temp_df_2.drop(temp_df_2.tail(1).index,inplace=True)
        temp_df_2.drop(temp_df_2.head(1).index,inplace=True)
        
        #if time changes --> check that the UCID also changes 
        if 
        
         

In [40]:
pivot_exercises.apply(lambda row: get_average_time(row), axis=1)

uuid
++5bdNp/LZvGenJ8Brp4n2SfS9d4pu4qA7cF7FQW7hk=    0
++9EkR6syMGk44XpyhOj40cg9xiXwCLS/TwEy+ujrL0=    0
++E4TrlDYvGtPBg1edhkLXLEEbnfiAgAamPQ33vpW8M=    0
++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=    1
++GobOSWqrsaxoRg1bMN+T6biIJcgBXwuOH/ddq3DiU=    1
                                               ..
zztwLsjf44Uzumou84qh04wObpugkIllLCIp/mgoMu8=    1
zzueVRV14tXIEgW4ixiqmtPNaFYz4MIifbMM/H2tmqA=    1
zzujjvdKzb4wL8cFKdF3Of5gg2XHzIDQCZ2TfaYpSNs=    1
zzvvUOwcSXXTXd8xoyimNcm2DjeVPLmZAI19WKfhSAM=    1
zzyukk9LNl7j/9xxgZj6roQBbGn1MjSPTCgc0r5sXpU=    1
Length: 72758, dtype: int64

In [41]:
pivot_exercises['avg_time_btwn_exercise'] = pivot_exercises.apply(lambda row: get_average_time(row), axis=1)
pivot_exercises

,ucid,avg_time_btwn_exercise
uuid,,
++5bdNp/LZvGenJ8Brp4n2SfS9d4pu4qA7cF7FQW7hk=,1,0
++9EkR6syMGk44XpyhOj40cg9xiXwCLS/TwEy+ujrL0=,1,0
++E4TrlDYvGtPBg1edhkLXLEEbnfiAgAamPQ33vpW8M=,1,0
++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,6,1
++GobOSWqrsaxoRg1bMN+T6biIJcgBXwuOH/ddq3DiU=,25,1
...,...,...
zztwLsjf44Uzumou84qh04wObpugkIllLCIp/mgoMu8=,5,1
zzueVRV14tXIEgW4ixiqmtPNaFYz4MIifbMM/H2tmqA=,4,1
zzujjvdKzb4wL8cFKdF3Of5gg2XHzIDQCZ2TfaYpSNs=,28,1


# Test and Check

In [11]:
log_problem.loc[log_problem['uuid'] == '++5bdNp/LZvGenJ8Brp4n2SfS9d4pu4qA7cF7FQW7hk=']

,timestamp_TW,uuid,ucid,upid,problem_number,exercise_problem_repeat_session,is_correct,total_sec_taken,total_attempt_cnt,used_hint_cnt,is_hint_used,is_downgrade,is_upgrade,level
5851364,2018-12-22 08:30:00 UTC,++5bdNp/LZvGenJ8Brp4n2SfS9d4pu4qA7cF7FQW7hk=,VY6aXT7f64ny+uy4pszHVNSy3WHyoFPuhwToxBhB3wM=,wgsncTN/ZG/jV1BVhOb0ExV10Y7nbx4HDXgpqfV98Ls=,2,1,True,17,1,0,False,NaN,NaN,0
10682047,2018-12-22 08:30:00 UTC,++5bdNp/LZvGenJ8Brp4n2SfS9d4pu4qA7cF7FQW7hk=,VY6aXT7f64ny+uy4pszHVNSy3WHyoFPuhwToxBhB3wM=,uIHmyJc0Ia07OXiS1k2wayrEA57AKPTypHaTYtKc4Po=,5,1,True,7,1,0,False,False,True,1
11828870,2018-12-22 08:30:00 UTC,++5bdNp/LZvGenJ8Brp4n2SfS9d4pu4qA7cF7FQW7hk=,VY6aXT7f64ny+uy4pszHVNSy3WHyoFPuhwToxBhB3wM=,SZt4MEfoLNcDjOc4qWBsd5Lw5M03DA0f22Baij04cbg=,1,1,True,193,1,0,False,NaN,NaN,0
13798435,2018-12-22 08:30:00 UTC,++5bdNp/LZvGenJ8Brp4n2SfS9d4pu4qA7cF7FQW7hk=,VY6aXT7f64ny+uy4pszHVNSy3WHyoFPuhwToxBhB3wM=,hrdzJpcOlqUZbvMB6A0P2fHVvn6US4bPm05s/PsjRi4=,3,1,True,55,1,0,False,NaN,NaN,0
15461504,2018-12-22 08:30:00 UTC,++5bdNp/LZvGenJ8Brp4n2SfS9d4pu4qA7cF7FQW7hk=,VY6aXT7f64ny+uy4pszHVNSy3WHyoFPuhwToxBhB3wM=,s2T1bZCtwOT2xgCGkHFRiqlSX8MHEOiW36tBJRXYj4E=,4,1,True,30,1,0,False,NaN,NaN,0


In [52]:
test_df = log_problem.loc[log_problem['uuid'] == '++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y='].sort_values(by=['timestamp_TW','ucid'])
test_df

,timestamp_TW,uuid,ucid,upid,problem_number,exercise_problem_repeat_session,is_correct,total_sec_taken,total_attempt_cnt,used_hint_cnt,is_hint_used,is_downgrade,is_upgrade,level
630946,2018-11-22 19:15:00 UTC,++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,eJs/Pql/Xui+9mjL0q4CwHI/r+w6Bn3tTCt0i6r4xrg=,IU8gI2g8wOudzcNuAYmLUsOsw6Vk0o9oCtbptXzyN5c=,1,1,False,147,2,3,True,NaN,NaN,0
8681983,2018-11-22 19:15:00 UTC,++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,hxF+GCZ2kFjuoHOUVj0a6Jn9NbRfydExscjLa0cvxuw=,TamfAzOmINKf88BC8JueWOyKXbQ3iHcXhQK6TNzvunc=,1,1,False,53,2,4,True,NaN,NaN,0
5741530,2018-11-22 19:30:00 UTC,++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,eJs/Pql/Xui+9mjL0q4CwHI/r+w6Bn3tTCt0i6r4xrg=,bPUH0Wg2TIKMw6TrkSmo4a6nIx7VJULUNCwPDDo1ROM=,4,1,False,16,2,3,True,NaN,NaN,0
5841177,2018-11-22 19:30:00 UTC,++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,eJs/Pql/Xui+9mjL0q4CwHI/r+w6Bn3tTCt0i6r4xrg=,i6EsuZyNTHaar8gTYbJi5ygFBt7fvKVXwV569lnsdu8=,2,1,False,122,1,3,True,NaN,NaN,0
11814698,2018-11-22 19:30:00 UTC,++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,eJs/Pql/Xui+9mjL0q4CwHI/r+w6Bn3tTCt0i6r4xrg=,/V2nyY10w/LwkgMURjwKiLZ6xxxH3EhLq2uDuuBpNPw=,3,1,False,12,1,4,True,NaN,NaN,0
465934,2018-11-22 19:30:00 UTC,++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,hxF+GCZ2kFjuoHOUVj0a6Jn9NbRfydExscjLa0cvxuw=,CcxsCZJLwy/bnXcIkFfQ9D+i/trRVXkZE04gOvVbA0Q=,3,1,False,10,1,5,True,NaN,NaN,0
736436,2018-11-22 19:30:00 UTC,++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,hxF+GCZ2kFjuoHOUVj0a6Jn9NbRfydExscjLa0cvxuw=,s5PXtw2Nybu+rIKBiTEF13gNrgVmVsC/ZkZ1Y29gkao=,4,1,False,7,1,3,True,NaN,NaN,0
2252765,2018-11-22 19:30:00 UTC,++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,hxF+GCZ2kFjuoHOUVj0a6Jn9NbRfydExscjLa0cvxuw=,mmfw96/hqjhgBZMBig0lN+PRoxBgtCEdliAdguLa2ck=,6,1,False,11,1,3,True,NaN,NaN,0
8681350,2018-11-22 19:30:00 UTC,++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,hxF+GCZ2kFjuoHOUVj0a6Jn9NbRfydExscjLa0cvxuw=,gSjTVenKrRWse5YP8uRgj4LRW5Y4bxcEDBRAV/EU9lI=,5,1,False,9,1,3,True,NaN,NaN,0
8736886,2018-11-22 19:30:00 UTC,++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,hxF+GCZ2kFjuoHOUVj0a6Jn9NbRfydExscjLa0cvxuw=,4dgBsDzDAxg/i9X7jQq+wEpBA2Vf4K4YOawg9zk/SX8=,7,1,False,34,3,4,True,NaN,NaN,0


In [61]:
test_df_2 = test_df.groupby('timestamp_TW').agg(['first', 'last']).stack().reset_index('timestamp_TW')

In [62]:
test_df_2['match'] = test_df_2.timestamp_TW.eq(test_df_2.timestamp_TW.shift())

In [64]:
test_df_2.inde

KeyError: 1

In [59]:
test_df_2.drop(test_df_2.tail(1).index,inplace=True)

In [60]:
test_df_2

,timestamp_TW,uuid,ucid,upid,problem_number,exercise_problem_repeat_session,is_correct,total_sec_taken,total_attempt_cnt,used_hint_cnt,is_hint_used,is_downgrade,is_upgrade,level,match
first,2018-11-22 19:15:00 UTC,++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,eJs/Pql/Xui+9mjL0q4CwHI/r+w6Bn3tTCt0i6r4xrg=,IU8gI2g8wOudzcNuAYmLUsOsw6Vk0o9oCtbptXzyN5c=,1,1,False,147,2,3,True,NaN,NaN,0,False
first,2018-11-22 19:30:00 UTC,++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,eJs/Pql/Xui+9mjL0q4CwHI/r+w6Bn3tTCt0i6r4xrg=,bPUH0Wg2TIKMw6TrkSmo4a6nIx7VJULUNCwPDDo1ROM=,4,1,False,16,2,3,True,NaN,NaN,0,False
first,2018-11-22 19:45:00 UTC,++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,m7rxdmifsJ8HxB91x1USFvW7op/ZusrvlAjq/FAmRE8=,hyguczdcRIhTjM698qM4/MY7UVveVNL/ZXvCgTHIHB4=,5,1,False,11,1,2,True,NaN,NaN,0,False
first,2018-12-02 19:45:00 UTC,++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,412PAnenNdYglQWXSlVtS1RYA7Yg60Wty166LCMaiHU=,JnLFiUhxhDMcQfPzNfSHjpTSHjLZkMs0vRwwMHTyS2c=,3,1,True,113,1,0,False,NaN,NaN,0,False
first,2018-12-02 20:15:00 UTC,++G4mkLfs4WDYhc1Ga+3G+/oqSniQQvLBm7SBQ3V39Y=,412PAnenNdYglQWXSlVtS1RYA7Yg60Wty166LCMaiHU=,sOVzAh48l7ojeTqa2/tAoJjJJ94/dIMb7a5itjqX7lk=,7,1,True,16,1,0,False,False,True,0,False
